<a href="https://colab.research.google.com/github/Amari-Lawal/Data-Science-Projects-/blob/main/NLP_Practice_Attempt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Upload a kaggle.json api key by going into settings of a kaggle account
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"stinner999palace","key":"473e7cf39093d8fb6cf2f7dd909f5633"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/ # Makes the directory to store the kaggle api key to be able to access the dataset
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d team-ai/spam-text-message-classification # Downloads dataset from kaggle

spam-text-message-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!ls

'kaggle (1).json'  'SPAM text message 20170820 - Data.csv'
 kaggle.json	    spam-text-message-classification.zip
 sample_data


In [5]:
import zipfile
zip_ref = zipfile.ZipFile("spam-text-message-classification.zip", "r") # Unzips the zip file
zip_ref.extractall()
zip_ref.close()

In [6]:
DATA_URL = "/content/SPAM text message 20170820 - Data.csv"
import pandas as pd
df = pd.read_csv(DATA_URL) # Loads in the 
display(df)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [7]:
import tensorflow as tf

In [8]:
import spacy
import pandas as pd
import numpy as np
#!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg') 
#nlp = spacy.load('en_core_web_sm')

In [9]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [11]:
# Get all sentences from the dataframe 
# Get tokens of words in each sentence
# Add to the vocab
# Then count frequency of words between all sentences
#parser = spacy.lang.en.English()
#parser = spacy.lang.en.English()
def tokenizer_extraction(dataset):
  # Imports Counter Function
  from collections import Counter
  my_tokens = nlp(dataset)
  # Creates set which you add the vocab to and makes sure that there are no duplicates
  vocab = set() # However this means the vocab won't be in order and the indexes may change, INDEXES is the data format we will use to input into model/ 
  # Gets the tokens from the document getting rid of stop words, punctuation, digits and inculding lemmas
  my_tokens = [token.lemma_.lower().strip() for token in my_tokens if not token.is_stop and not token.is_punct and not token.is_digit and not token.is_currency and token.is_ascii] # May want to get rid of non unicode values with ord(token) < 128 on ascii table.
  # Adds the tokens in words to vocab set.
  for i in my_tokens:
    vocab.add(i)
  # Counts the word frequency 
  word_freq = Counter(my_tokens)
  common_words = word_freq.most_common(len(dataset))


  print(f'Vocab Size: {len(vocab)} ')
  print(f'Vocab Words: {vocab}')
  print("Word Frequency:",common_words)
  return my_tokens

In [12]:
#vocab_lis, word_lis, words = tokenizer_extraction(df,1)

In [13]:
class predictors(TransformerMixin): # 
  def transform(self,X,**transform_params):
    # Cleaning Text
    return [clean_text(text) for text in X]
  def fit(self,X,y=None,**fit_params):
    return self
    def get_params(self,deep=True):
      return {}
def clean_text(text):
    return text.strip().lower()

In [14]:
bow_vector = CountVectorizer(tokenizer = tokenizer_extraction,ngram_range=(1,1))

In [15]:
tfidf_vector = TfidfVectorizer(tokenizer=tokenizer_extraction)

In [16]:
from sklearn.model_selection import train_test_split
X = df["Message"]
df_dum = pd.get_dummies(df["Category"])
ylabels = df_dum['spam'] # Set values as dummy variables
X_train, X_test, y_train, y_test = train_test_split(X,ylabels,test_size=0.3)

In [17]:
print(ylabels)
# 747 spam

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: spam, Length: 5572, dtype: uint8


In [18]:
from tensorflow.keras.layers import *
from tensorflow.keras import Input

In [19]:
print(X_train.shape)
display(pd.DataFrame(X_train))
# Number of Rows or sentences is 3900

(3900,)


,Message
3397,URGENT! Your Mobile number has been awarded wi...
3994,Nimbomsons. Yep phone knows that one. Obviousl...
2087,Dude how do you like the buff wind.
4241,The LAY MAN! Just to let you know you are miss...
2689,Yes I know the cheesy songs from frosty the sn...
...,...
662,Then mum's repent how?
5119,Lol for real. She told my dad I have cancer
5305,Hi missed your Call and my mumHas beendropping...
2939,Hey i've booked the pilates and yoga lesson al...


In [21]:
# Creates Pipeline for 
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
# wrap the model using the function you created
#clf = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=create_model,verbose=0)

# When using Pipeline step has to have fit or fit_transform in the objects inserted. By using Transformer MixIn
pipe = Pipeline([("cleaner",predictors()),
                 ("vectorizer",bow_vector),
                 ("classifier",classifier)
                 ])
pipe.fit(X_train,y_train)


Streaming output truncated to the last 5000 lines.
Vocab Words: {'challenge', 'know'}
Word Frequency: [('challenge', 1), ('know', 1)]
Vocab Size: 13 
Vocab Words: {'flurry', 'snow', 'usually', 'go', 'wish', 'think', 'hit', 'ground', 'lt;#&gt', 'melt', 'bear', 'eek', 'to'}
Word Frequency: [('snow', 2), ('wish', 1), ('think', 1), ('go', 1), ('to', 1), ('flurry', 1), ('usually', 1), ('melt', 1), ('hit', 1), ('ground', 1), ('eek', 1), ('lt;#&gt', 1), ('bear', 1)]
Vocab Size: 5 
Vocab Words: {'ok', 'call', 'instead', 'fun', 'mom'}
Word Frequency: [('ok', 1), ('call', 1), ('mom', 1), ('instead', 1), ('fun', 1)]
Vocab Size: 2 
Vocab Words: {'juliana', 'want'}
Word Frequency: [('juliana', 1), ('want', 1)]
Vocab Size: 3 
Vocab Words: {'ok', 'mind', 'neva'}
Word Frequency: [('neva', 1), ('mind', 1), ('ok', 1)]
Vocab Size: 10 
Vocab Words: {'www.07781482378.com', 'term', 'reply', 'message', 'text', 'condition', 'activate', 'word', 'free', 'visit'}
Word Frequency: [('free', 3), ('message', 3), ('a

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x7fe30182d550>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 t...\\w+\\b',
                                 tokenizer=<function tokenizer_extraction at 0x7fe301882dd0>,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
             

In [22]:
from sklearn import metrics
predicted = pipe.predict(X_test)
print(f'-------------------------------------------------------------------------')
print(f'Logistsic Regression Accuracy: {metrics.accuracy_score(y_test,predicted)}')
print(f'Logistic Regression Precision: {metrics.precision_score(y_test,predicted)}')
print(f'Logistic Regression Recall: {metrics.recall_score(y_test,predicted)}')

Streaming output truncated to the last 5000 lines.
Word Frequency: [('stop', 2), ('natalja', 1), ('f', 1), ('invite', 1), ('friend', 1), ('reply', 1), ('yes-440', 1), ('no-440', 1), ('www.sms.ac/u/nat27081980', 1), ('send', 1), ('frnd', 1)]
Vocab Size: 8 
Vocab Words: {'hair', 'day', 'ur', 'call', 'sense', 'cut', 'noe', 'paragon'}
Word Frequency: [('hair', 3), ('cut', 2), ('day', 1), ('ur', 1), ('paragon', 1), ('call', 1), ('sense', 1), ('noe', 1)]
Vocab Size: 12 
Vocab Words: {'homeowner', 'previously', 'welcome', 'refuse', 'text', 'help', 'loan', 'tenant', '75,000', 'free', 'purpose', '+'}
Word Frequency: [('help', 2), ('loan', 1), ('purpose', 1), ('75,000', 1), ('homeowner', 1), ('+', 1), ('tenant', 1), ('welcome', 1), ('previously', 1), ('refuse', 1), ('free', 1), ('text', 1)]
Vocab Size: 6 
Vocab Words: {'couple', 'time', '4u', 'awesome', 'haha', 'come'}
Word Frequency: [('haha', 1), ('awesome', 1), ('4u', 1), ('couple', 1), ('time', 1), ('come', 1)]
Vocab Size: 5 
Vocab Words: {'

In [23]:
for i in range(0,len(X_test)):
  if predicted[i] == 1:
    print(f'Spam: {X_test.iloc[i]}')
  elif predicted[i] == 0:
    print(f'Not Spam: {X_test.iloc[i]}')


Not Spam: No:-)i got rumour that you going to buy apartment in chennai:-)
Not Spam: I not busy juz dun wan 2 go so early.. Hee..
Not Spam: Erutupalam thandiyachu
Not Spam: I cant wait for cornwall. Hope tonight isnt too bad as well but its rock night shite. Anyway im going for a kip now have a good night. Speak to you soon.
Not Spam: Tell dear what happen to you. Why you talking to me like an alian
Not Spam: Message:some text missing* Sender:Name Missing* *Number Missing *Sent:Date missing *Missing U a lot thats y everything is missing sent via fullonsms.com
Spam: Natalja (25/F) is inviting you to be her friend. Reply YES-440 or NO-440 See her: www.SMS.ac/u/nat27081980 STOP? Send STOP FRND to 62468
Not Spam: That day ü say ü cut ur hair at paragon, is it called hair sense? Do ü noe how much is a hair cut?
Spam: Loan for any purpose £500 - £75,000. Homeowners + Tenants welcome. Have you been previously refused? We can still help. Call Free 0800 1956669 or text back 'help'
Not Spam: 

In [51]:
def text_predict(input):
  #text = list()
  text = []
  text.append(input)
  pred_text = pipe.predict(text)
  if pred_text[0] == 1:
    print(f'--------------------------------------------------------------------------')
    print(f'Spam: {text[0]}')
  elif pred_text[0] == 0:
    print(f'---------------------------------------------------------------------------')
    print(f'Not Spam: {text[0]}')

text_predict(input())
#list(input())

Claim your free car now. To claim free car call 09567843215
Vocab Size: 3 
Vocab Words: {'claim', 'car', 'free'}
Word Frequency: [('claim', 2), ('free', 2), ('car', 2)]
--------------------------------------------------------------------------
Spam: Claim your free car now. To claim free car call 09567843215


TODO LSTM Network
1. Next sequence data without vectorizing
2. Feed into Word Embedding then Neural Network

In [ ]:
# For padding sequence object for pipeline make

#class sequencing(TransformerMixin):
#  def transform(self,X,**transform_params):
#    # Cleaning Text
#    return lambda sequence: #Then sequence data to feed into LSTM 
#  def fit(self,X,y=None,**fit_params):
#    return self
#    def get_params(self,deep=True):
#      return {}
#def or have Sequence function(text):
#    return text.strip().lower()

In [52]:
#def Lstm():
#    inputs = Input(name='inputs',shape=[max_len])
#    layer = Embedding(max_words,50,input_length=max_len)(inputs)  #vocab, vector space ;output of the Embedding layer is a 2D vector
##    layer = LSTM(64)(layer)
 #   layer = Dense(256,name='FC1')(layer)
 #   layer = Activation('relu')(layer)
 #   layer = Dropout(0.5)(layer)
 #   layer = Dense(1,name='out_layer')(layer)
#    layer = Activation('sigmoid')(layer)
#    model = Model(inputs=inputs,outputs=layer)
#    return model

In [ ]:
#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing import sequence
#from keras.utils import to_categorical
#from keras.callbacks import EarlyStopping

#max_words = 1000
#max_len = 150
#tok = Tokenizer(num_words=max_words)
#tok.fit_on_texts(X_train)

#sequences = tok.texts_to_sequences(X_train)
#sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

#max_words = 1000
#max_len = 150
#tf.keras.preprocessing.sequence.pad_sequences(X_train,maxlen=150)
#tok = Tokenizer(num_words=max_words)
#tok.fit_on_texts(X_train)


# X_train Transformations from cleaning data from predictors
#pred_train = predictors()
#X_train = pred_train.transform(X_train)
#X_train = bow_vector.fit_transform(X_train)

#X_test = pred_train.transform(X_test)
#X_test = bow_vector.fit_transform(X_test)

